<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/02_ROCKET_a_new_SOTA_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: timeseriesAI@gmail.com

## Purpose 😇

The purpose of this notebook is to introduce you to Rocket. 

ROCKET (RandOm Convolutional KErnel Transform) is a new Time Series Classification (TSC) method that has just been released (Oct 29th, 2019), and has achieved **state-of-the-art performance on the UCR univariate time series classification datasets, surpassing HIVE-COTE (the previous state of the art since 2017) in accuracy, with exceptional speed compared to other traditional DL methods.** 

To achieve these 2 things at once is **VERY IMPRESSIVE**. ROCKET is certainly a new TSC method you should try.

Authors:
Dempster, A., Petitjean, F., & Webb, G. I. (2019). ROCKET: Exceptionally fast and accurate time series classification using random convolutional kernels. arXiv preprint arXiv:1910.13051.

[paper](https://arxiv.org/pdf/1910.13051)

There are 2 main limitations to the original ROCKET method though:
- Released code doesn't handle multivariate data
- It doesn't run on a GPU, so it's slow when used with a large datasets

In this notebook you will learn: 
- how you can use the original ROCKET method
- you will also learn about a new ROCKET version I have developed in Pytorch, that handles both **univariate and multivariate** data, and uses **GPU**
- you will see how you can integrate the ROCKET features with fastai or other classifiers

## Import libraries 📚

In [ ]:
# ## NOTE: UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI
# stable = False # True: stable version in pip, False: latest version from github
# if stable: 
#     !pip install tsai -U >> /dev/null
# else:      
#     !pip install git+https://github.com/timeseriesAI/tsai.git -U >> /dev/null
# ## NOTE: REMEMBER TO RESTART (NOT RECONNECT/ RESET) THE KERNEL/ RUNTIME ONCE THE INSTALLATION IS FINISHED

In [2]:
import numpy as np

In [25]:
from tsai.all import *
computer_setup()

os             : Linux-5.4.0-80-generic-x86_64-with-debian-bullseye-sid
python         : 3.7.3
tsai           : 0.2.22
fastai         : 2.5.2
fastcore       : 1.3.26
torch          : 1.9.1+cu102
n_cpus         : 10
device         : cuda (Tesla T4)


## How to use the original ROCKET method? 🚀

ROCKET is applied in 2 phases:

1. Generate features from each time series: ROCKET calculates 20k features from each time series, independently of the sequence length. 
2. Apply a classifier to those calculated features. Those features are then used by the classifier of your choice. In the original code they use 2 simple linear classifiers: RidgeClassifierCV and Logistic Regression, but you can use any classifier.

### 1️⃣ Generate features

Let's first generate the features. We'll import data from a UCR Time Series dataset.

The original method requires the time series to be in a 2d array of shape (samples, len). Remember than only univariate sequences are allow in this original method.

In [20]:
data_p1 = np.load("../Pre-Processing/trials/subject_1_session_1_ICA_CAR.npy")
labels_p1 = np.array([0,1,2, 3]*5)

data_p2 = np.load("../Pre-Processing/trials/subject_")

## How to use ROCKET with large and/ or multivariate datasets on GPU? - Recommended ⭐️

As stated before, the current ROCKET method doesn't support multivariate time series or GPU. This may be a drawback in some cases. 

To overcome both limitations I've created a multivariate ROCKET on GPU in Pytorch. 

### 1️⃣ Generate features

First you prepare the input data and normalize it per sample. The input to ROCKET Pytorch is a 3d tensor of shape (samples, vars, len), preferrable on gpu.

The way to use ROCKET in Pytorch is the following:

* Create a dataset as you would normally do in `tsai`. 
* Create a TSDataLoaders with the following kwargs: 
    * drop_last=False. In this way we get features for every input sample.
    * shuffle_train=False
    * batch_tfms=[TSStandardize(by_sample=True)] so that input is normalized by sample, as recommended by the authors


In [26]:
#X, y, splits = get_UCR_data('HandMovementDirection', split_data=False)
#splits = RandomSplitter()(range_of(data_p1))

splits = TrainValidTestSplitter(stratify = True, random_state= 0)(range_of(data_p1))
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_sample=True)]
dls = get_ts_dls(data_p1, labels_p1,splits = splits, tfms=tfms, drop_last=False, shuffle_train=False, batch_tfms=batch_tfms, bs=10_000)

In [27]:
splits

((#16) [10,17,6,13,4,2,5,14,9,7...], (#4) [18,1,19,8])

☣️☣️ You will be able to create a dls (TSDataLoaders) object with unusually large batch sizes. I've tested it with a large dataset and a batch size = 100_000 and it worked fine. This is because ROCKET is not a usual Deep Learning model. It just applies convolutions (kernels) one at a time to create the features.

Instantiate a rocket model with the desired n_kernels (authors use 10_000) and kernel sizes (7, 9 and 11 in the original paper). 

In [28]:
model = build_ts_model(ROCKET, dls=dls, n_kernels = 20000, kss = [3, 5, 7, 9, 11, 13]) # n_kernels=10_000, kss=[7, 9, 11] set by default, but you can pass other values as kwargs

Now generate rocket features for the entire train and valid datasets using the create_rocket_features convenience function `create_rocket_features`.

And we now transform the original data, creating 20k features per sample

In [29]:
X_train, y_train = create_rocket_features(dls.train, model)
X_valid, y_valid = create_rocket_features(dls.valid, model)
X_train.shape, X_valid.shape

((16, 40000), (4, 40000))

### 2️⃣ Apply a classifier

Once you build the 20k features per sample, you can use them to train any classifier of your choice.

#### RidgeClassifierCV

And now you apply a classifier of your choice. 
With RidgeClassifierCV in particular, there's no need to normalize the calculated features before passing them to the classifier, as it does it internally (if normalize is set to True as recommended by the authors).

In [30]:
from sklearn.linear_model import RidgeClassifierCV
ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train, y_train)
print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')

alpha: 1.00E-08  train: 1.00000  valid: 0.25000


This result is amazing!! The previous state of the art (Inceptiontime) was .37837

#### Logistic Regression

In the case of other classifiers (like Logistic Regression), the authors recommend a per-feature normalization.

In [32]:
eps = 1e-5
Cs = np.logspace(-5, 5, 11)
from sklearn.linear_model import LogisticRegression
best_loss = np.inf
for i, C in enumerate(Cs):
    f_mean = X_train.mean(axis=0, keepdims=True)
    f_std = X_train.std(axis=0, keepdims=True) + eps  # epsilon to avoid dividing by 0
    X_train_tfm2 = (X_train - f_mean) / f_std
    X_valid_tfm2 = (X_valid - f_mean) / f_std
    classifier = LogisticRegression(penalty='l2', C=C, n_jobs=-1)
    classifier.fit(X_train_tfm2, y_train)
    probas = classifier.predict_proba(X_train_tfm2)
    loss = nn.CrossEntropyLoss()(torch.tensor(probas), torch.tensor(y_train)).item()
    train_score = classifier.score(X_train_tfm2, y_train)
    val_score = classifier.score(X_valid_tfm2, y_valid)
    if loss < best_loss:
        best_eps = eps
        best_C = C
        best_loss = loss
        best_train_score = train_score
        best_val_score = val_score
    print('{:2} eps: {:.2E}  C: {:.2E}  loss: {:.5f}  train_acc: {:.5f}  valid_acc: {:.5f}'.format(
        i, eps, C, loss, train_score, val_score))
print('\nBest result:')
print('eps: {:.2E}  C: {:.2E}  train_loss: {:.5f}  train_acc: {:.5f}  valid_acc: {:.5f}'.format(
        best_eps, best_C, best_loss, best_train_score, best_val_score))

 0 eps: 1.00E-05  C: 1.00E-05  loss: 1.31095  train_acc: 1.00000  valid_acc: 0.25000
 1 eps: 1.00E-05  C: 1.00E-04  loss: 1.02247  train_acc: 1.00000  valid_acc: 0.25000
 2 eps: 1.00E-05  C: 1.00E-03  loss: 0.80026  train_acc: 1.00000  valid_acc: 0.25000
 3 eps: 1.00E-05  C: 1.00E-02  loss: 0.75231  train_acc: 1.00000  valid_acc: 0.25000
 4 eps: 1.00E-05  C: 1.00E-01  loss: 0.74485  train_acc: 1.00000  valid_acc: 0.25000
 5 eps: 1.00E-05  C: 1.00E+00  loss: 0.74382  train_acc: 1.00000  valid_acc: 0.25000
 6 eps: 1.00E-05  C: 1.00E+01  loss: 0.74369  train_acc: 1.00000  valid_acc: 0.25000
 7 eps: 1.00E-05  C: 1.00E+02  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.25000
 8 eps: 1.00E-05  C: 1.00E+03  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.25000
 9 eps: 1.00E-05  C: 1.00E+04  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.25000
10 eps: 1.00E-05  C: 1.00E+05  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.25000

Best result:
eps: 1.00E-05  C: 1.00E+02  train_loss: 0.74367  tr

☣️ Note: Epsilon has a large impact on the result. You can actually test several values to find the one that best fits your problem, but bear in mind you can only select C and epsilon based on train data!!! 

##### RandomSearch

One way to do this would be to perform a random search using several epsilon and C values

In [33]:
n_tests = 10
epss = np.logspace(-8, 0, 9)
Cs = np.logspace(-5, 5, 11)

from sklearn.linear_model import LogisticRegression
best_loss = np.inf
for i in range(n_tests):
    eps = np.random.choice(epss)
    C = np.random.choice(Cs)
    f_mean = X_train.mean(axis=0, keepdims=True)
    f_std = X_train.std(axis=0, keepdims=True) + eps  # epsilon
    X_train_tfm2 = (X_train - f_mean) / f_std
    X_valid_tfm2 = (X_valid - f_mean) / f_std
    classifier = LogisticRegression(penalty='l2', C=C, n_jobs=-1)
    classifier.fit(X_train_tfm2, y_train)
    probas = classifier.predict_proba(X_train_tfm2)
    loss = nn.CrossEntropyLoss()(torch.tensor(probas), torch.tensor(y_train)).item()
    train_score = classifier.score(X_train_tfm2, y_train)
    val_score = classifier.score(X_valid_tfm2, y_valid)
    if loss < best_loss:
        best_eps = eps
        best_C = C
        best_loss = loss
        best_train_score = train_score
        best_val_score = val_score
    print('{:2}  eps: {:.2E}  C: {:.2E}  loss: {:.5f}  train_acc: {:.5f}  valid_acc: {:.5f}'.format(
        i, eps, C, loss, train_score, val_score))
print('\nBest result:')
print('eps: {:.2E}  C: {:.2E}  train_loss: {:.5f}  train_acc: {:.5f}  valid_acc: {:.5f}'.format(
        best_eps, best_C, best_loss, best_train_score, best_val_score))

 0  eps: 1.00E-01  C: 1.00E+02  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.50000
 1  eps: 1.00E-01  C: 1.00E+01  loss: 0.74372  train_acc: 1.00000  valid_acc: 0.50000
 2  eps: 1.00E-03  C: 1.00E-04  loss: 1.03652  train_acc: 1.00000  valid_acc: 0.25000
 3  eps: 1.00E-05  C: 1.00E-02  loss: 0.75231  train_acc: 1.00000  valid_acc: 0.25000
 4  eps: 1.00E+00  C: 1.00E+01  loss: 0.74374  train_acc: 1.00000  valid_acc: 0.50000
 5  eps: 1.00E+00  C: 1.00E-05  loss: 1.36144  train_acc: 1.00000  valid_acc: 0.50000
 6  eps: 1.00E+00  C: 1.00E+04  loss: 0.74367  train_acc: 1.00000  valid_acc: 0.50000
 7  eps: 1.00E-08  C: 1.00E-01  loss: 0.74485  train_acc: 1.00000  valid_acc: 0.25000
 8  eps: 1.00E-05  C: 1.00E-02  loss: 0.75231  train_acc: 1.00000  valid_acc: 0.25000
 9  eps: 1.00E-04  C: 1.00E+00  loss: 0.74382  train_acc: 1.00000  valid_acc: 0.25000

Best result:
eps: 1.00E+00  C: 1.00E+04  train_loss: 0.74367  train_acc: 1.00000  valid_acc: 0.50000


In general, the original method may be a bit faster than the GPU method, but for larger datasets, there's a great benefit in using the GPU version.

In addition to this, I have also run the code on the TSC UCR multivariate datasets (all the ones that don't contain nan values), and the results are also very good, beating the previous state-of-the-art in this category as well by a large margin. For example, ROCKET reduces InceptionTime errors by 26% on average.

#### XGBoost

In [34]:
eps = 1e-6

# normalize 'per feature'
f_mean = X_train.mean(axis=0, keepdims=True)
f_std = X_train.std(axis=0, keepdims=True) + eps
X_train_norm = (X_train - f_mean) / f_std
X_valid_norm = (X_valid - f_mean) / f_std

import xgboost as xgb
classifier = xgb.XGBClassifier(max_depth=3,
                               learning_rate=0.1,
                               n_estimators=100,
                               verbosity=1,
                               objective='binary:logistic',
                               booster='gbtree',
                               tree_method='auto',
                               n_jobs=-1,
                               gpu_id=default_device().index,
                               gamma=0,
                               min_child_weight=1,
                               max_delta_step=0,
                               subsample=.5,
                               colsample_bytree=1,
                               colsample_bylevel=1,
                               colsample_bynode=1,
                               reg_alpha=0,
                               reg_lambda=1,
                               scale_pos_weight=1,
                               base_score=0.5,
                               random_state=0,
                               missing=None)

classifier.fit(X_train_norm, y_train)
preds = classifier.predict(X_valid_norm)
(preds == y_valid).mean()

[15:44:20] WARNING: ../src/learner.cc:573: 
Parameters: { "scale_pos_weight" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:44:21] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBoostError: [15:44:23] ../src/c_api/c_api_utils.h:161: Invalid missing value: null
Stack trace:
  [bt] (0) /opt/tljh/user/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9133f) [0x7fccfd8c633f]
  [bt] (1) /opt/tljh/user/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9c811) [0x7fccfd8d1811]
  [bt] (2) /opt/tljh/user/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(+0x9cac0) [0x7fccfd8d1ac0]
  [bt] (3) /opt/tljh/user/lib/python3.7/site-packages/xgboost/lib/libxgboost.so(XGBoosterPredictFromDense+0x225) [0x7fccfd8b9005]
  [bt] (4) /opt/tljh/user/lib/python3.7/lib-dynload/../../libffi.so.6(ffi_call_unix64+0x4c) [0x7fcebaed9ec0]
  [bt] (5) /opt/tljh/user/lib/python3.7/lib-dynload/../../libffi.so.6(ffi_call+0x22d) [0x7fcebaed987d]
  [bt] (6) /opt/tljh/user/lib/python3.7/lib-dynload/_ctypes.cpython-37m-x86_64-linux-gnu.so(_ctypes_callproc+0x2ce) [0x7fcebbed6f7e]
  [bt] (7) /opt/tljh/user/lib/python3.7/lib-dynload/_ctypes.cpython-37m-x86_64-linux-gnu.so(+0x139b4) [0x7

## Conclusions

ROCKET is a great method for TSC that has established a new level of performance both in terms of accuracy and time. It does it by successfully applying an approach quite different from the traditional DL approaches. The method uses 10k random kernels to generate features that are then classified by linear classifiers (although you may use a classifier of your choice).
The original method has 2 limitations (lack of multivariate and lack of GPU support) that are overcome by the Pytorch implementation shared in this notebook.

So this is all the code you need to train a state-of-the-art model using rocket and GPU in `tsai`:

```
X, y, splits = get_UCR_data('HandMovementDirection', return_split=False)
tfms  = [None, [Categorize()]]
batch_tfms = [TSStandardize(by_sample=True)]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=64, drop_last=False, shuffle_train=False, batch_tfms=[TSStandardize(by_sample=True)])
model = create_model(ROCKET, dls=dls)
X_train, y_train = create_rocket_features(dls.train, model)
X_valid, y_valid = create_rocket_features(dls.valid, model)
ridge = RidgeClassifierCV(alphas=np.logspace(-8, 8, 17), normalize=True)
ridge.fit(X_train, y_train)
print(f'alpha: {ridge.alpha_:.2E}  train: {ridge.score(X_train, y_train):.5f}  valid: {ridge.score(X_valid, y_valid):.5f}')
```